In [58]:
import torch
import random
import zipfile
import time
import math
import numpy as np
from torch import nn, optim
import torch.nn.functional as F

In [59]:
def load_data_jay_lyrics():
    with zipfile.ZipFile('./Dataset/JayLyrics/jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars = f.read().decode('utf-8')
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx_dict = dict([(char, i) for i, char in enumerate(idx_to_char)])
    idx_to_char_dict = dict([(i, char) for i, char in enumerate(idx_to_char)])
    corpus_indice = [char_to_idx_dict[char] for char in corpus_chars]
    return corpus_chars, corpus_indice, idx_to_char_dict, char_to_idx_dict, len(
        idx_to_char_dict)


corpus_chars, corpus_indice, idx_to_char_dict, char_to_idx_dict, vocab_size = load_data_jay_lyrics()

In [60]:
def grad_clipping(params, theta):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    norm = torch.tensor([0.0], device = device)
    for param in params:
        norm += (param.grad.data**2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta/norm) 

In [61]:
def data_iter_random(corpus_indice, batch_size, num_step, device = None):
    num_example = (len(corpus_indice)-1)//num_step
    epoch_size = num_example//batch_size
    example_indice = list(range(num_example))
    random.shuffle(example_indice)
    
    def _data(pos):
        return corpus_indice[pos:pos+num_step]
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    for i in range(epoch_size):
        i = i*batch_size
        X = [_data(j*num_step) for j in example_indice[i:i+batch_size]]
        Y = [_data(j*num_step+1) for j in example_indice[i: i+batch_size]]
        yield torch.tensor(X, dtype = torch.float, device = device), torch.tensor(Y, dtype = torch.float, device = device)

In [62]:
def data_iter_consecutive(corpus_indice, batch_size, num_step, device = None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_indice = torch.tensor(corpus_indice, dtype = torch.float, device = device)
    data_len = len(corpus_indice)
    batch_len = data_len//batch_size
    indice = corpus_indice[0:batch_len*batch_size].view(batch_size, batch_len)
    epoch_size = batch_len//num_step
    
    for i in range(epoch_size):
        i = i*num_step
        X = indice[:, i: i+num_step]
        X1 = indice[:, i+1:i+num_step+1]
        yield X, X1

In [63]:
class RnnModel(nn.Module):
    def __init__(self, vocab_size, num_hiddens, num_layers):
        super().__init__()
        self.rnn_layer = nn.RNN(input_size=vocab_size, hidden_size=num_hiddens, num_layers=num_layers)
        self.hidden_size = num_hiddens
        self.vocab_size = vocab_size
        self.fc_layer = nn.Linear(num_hiddens, vocab_size)
        self.state = None
        
    def forward(self, inputs, state):
        X = self.to_onehot(inputs, self.vocab_size)
        Y, self.state = self.rnn_layer(X, state)              #n*b*v, n*b*h
        output = self.fc_layer(Y.view(-1, Y.shape[-1]))
        return output, self.state
    
    @staticmethod
    def to_onehot(X, n_class):
        def one_hot(x,n_class):
            x = x.long()
            res=torch.zeros(x.shape[0],n_class, dtype = torch.float, device = x.device)
            res.scatter_(1,x.view(-1,1),1)
            return res        
        return torch.stack([one_hot(X[:,i],n_class) for i in range(X.shape[1])])   #n*b*v

In [64]:
class GruRnnModel(nn.Module):
    def __init__(self, vocab_size, num_hiddens, num_layers):
        super().__init__()
        self.gru_layer = nn.GRU(input_size=vocab_size, hidden_size=num_hiddens, num_layers=num_layers)
        self.hidden_size = num_hiddens
        self.vocab_size = vocab_size
        self.fc_layer = nn.Linear(num_hiddens, vocab_size)
        self.state = None
        
    def forward(self, inputs, state):
        X = self.to_onehot(inputs, self.vocab_size)
        H, self.state = self.gru_layer(X, state)              #n*b*v, n*b*h
        Y = self.fc_layer(H.view(-1, Y.shape[-1]))
        return Y, self.state
    
    @staticmethod
    def to_onehot(X, n_class):
        def one_hot(x,n_class):
            x = x.long()
            res=torch.zeros(x.shape[0],n_class, dtype = torch.float, device = x.device)
            res.scatter_(1,x.view(-1,1),1)
            return res        
        return torch.stack([one_hot(X[:,i],n_class) for i in range(X.shape[1])]) 

In [65]:
class LstmRnnModel(nn.Module):
    def __init__(self, vocab_size, num_hiddens, num_layers):
        super().__init__()
        self.lstm_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens, num_layers=num_layers)
        self.hidden_size = num_hiddens
        self.vocab_size = vocab_size
        self.fc_layer = nn.Linear(self.hidden_size, vocab_size)
        self.state = None
        
    def forward(self, inputs, state):
        X = self.to_onehot(inputs, self.vocab_size)
        H, self.state = self.lstm_layer(X, state)              #n*b*v, n*b*h
        Y = self.fc_layer(H.view(-1, H.shape[-1]))
        return Y, self.state
    
    @staticmethod
    def to_onehot(X, n_class):
        def one_hot(x,n_class):
            x = x.long()
            res=torch.zeros(x.shape[0],n_class, dtype = torch.float, device = x.device)
            res.scatter_(1,x.view(-1,1),1)
            return res        
        return torch.stack([one_hot(X[:,i],n_class) for i in range(X.shape[1])]) 

In [66]:
def predict_rnn(prefix, num_chars, model,idx_to_char_dict, char_to_ida_dict, state = None):
    vocab_size = len(idx_to_char_dict)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    output = [prefix[0]]
    for t in range(num_chars+len(prefix)-1):
        X_indice = torch.tensor([char_to_idx_dict[output[-1]]], device = device).view(-1,1)
        Y, state = model(X_indice, state)
        if t<len(prefix)-1: 
            output.append(prefix[t+1])
        else:    
            output.append(idx_to_char_dict[int(Y.argmax(dim = 1).item())])
    return ''.join(output)

In [67]:
def train_and_predict(corpus_indice, data_iter, batch_size, num_steps, 
                      num_epoches, model, lr, clipping_theta,
                      idx_to_char_dict, char_to_idx_dict, 
                      prefixes, num_chars):
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    vocab_size = len(idx_to_char_dict)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(),lr=lr)
    state = None
    
    for epoch in range(num_epoches): 
        train_loss_sum, n = 0.0, 0
        for X_indice_list, Y_indice_list in data_iter(corpus_indice, batch_size, num_steps, device):
            if state is not None:
                state = (state[0].detach(), state[1].detach())
            Y_hat, state = model(X_indice_list, state) 
            Y_indice_list = torch.transpose(Y_indice_list, 0, 1).contiguous().view(-1).long()
            loss = loss_fn(Y_hat, Y_indice_list)
            optimizer.zero_grad()
            loss.backward()
            grad_clipping(model.parameters(), clipping_theta)
            optimizer.step()
            train_loss_sum += loss.item()
            n += 1
        if (epoch+1)%50 == 0:
            train_loss = train_loss_sum/n
            try:
                perplexity = math.exp(train_loss)
            except OverflowError:
                perplexity = float('inf')
            print('epoch: %d, train_loss: %.2f, perplexity: %f'%(epoch+1, train_loss, perplexity))
            
            for prefix in prefixes:
                print(predict_rnn(prefix, num_chars, model, idx_to_char_dict, char_to_idx_dict, state = None))
                

In [71]:
num_epoches, batch_size, lr, clipping_theta = 250, 35, 1e-2, 1e-2
num_steps, num_chars, prefixes = 50, 50, ['后知后觉']
#model = RnnModel(vocab_size, 256, 1).to('cuda')
#model = GruRnnModel(vocab_size,256,1).to('cuda') 
model = LstmRnnModel(vocab_size, 256, 2).to('cuda')

In [72]:
train_and_predict(corpus_indice, data_iter_consecutive, batch_size, num_steps, 
                      num_epoches, model, lr, clipping_theta,
                      idx_to_char_dict, char_to_idx_dict, 
                      prefixes, num_chars)

epoch: 50, train_loss: 3.78, perplexity: 43.609551
后知后觉 我有我不我 我不你 我 我不想我不你 我不你 我不你 我不你 我不你 我不你 我不你 我不你 我不
epoch: 100, train_loss: 0.95, perplexity: 2.573015
后知后觉 我该好好生活 一颗两颗三颗四颗 连成线背著背默默许下心愿 祭司 神殿 征战 弓箭 是谁的从前 喜欢
epoch: 150, train_loss: 0.09, perplexity: 1.090078
后知后觉  该到纵 是那么考倒一身笑过天对我感动的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女
epoch: 200, train_loss: 0.03, perplexity: 1.029071
后知后觉 你已经痛开始的时色蜡烛 温暖了空屋 白色蜡烛 温暖了空屋藤蔓植物 爬满了伯爵的坟墓 古堡里一片荒芜
epoch: 250, train_loss: 0.03, perplexity: 1.025524
后知后觉  感动看 一个人慢慢的回忆 没有了过去 我将往事抽离 如果我遇见你是一场悲剧 我可以让生命就这样毫
